In [267]:
from stem import Signal
from stem.control import Controller
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
from chromedriver_py import binary_path
import csv
import hashlib
import codecs
import pprint
import random
import re
import requests
import sys
import time

In [268]:
_HOST = 'https://scholar.google.com'
_AUTHSEARCH = '/citations?view_op=search_authors&hl=en&mauthors={0}'
_CITATIONAUTH = '/citations?user={0}&hl=en'
_CITATIONPUB = '/citations?view_op=view_citation&citation_for_view={0}'


In [251]:
def switchIP(): #switch IP
    with Controller.from_port(port = 9051) as controller:
        controller.authenticate()
        controller.signal(Signal.NEWNYM)

def get_browser(PROXY = None): #get browser and use proxy
    chrome_options = webdriver.ChromeOptions()
    if PROXY != None:
        chrome_options.add_argument('--proxy-server=SOCKS5://{0}'.format(PROXY)) 
    chrome_options.add_argument('blink-settings=imagesEnabled=false') # don't load pictures.
    chrome_options.add_argument('--headless') #don't launch visibile browser interface
    executable_path='/Users/fewave/project/python/demo/chromedriver' # set browser driver path
    return webdriver.Chrome(executable_path=binary_path, options=chrome_options)


In [256]:
def get_soup(url):
    switchIP() 
    driver = get_browser()
    driver.get(url)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    return soup

def get_next_page(url): #to automatically turn the page
    time.sleep(5) #wait 5 seconds to avoiding web-crawler detector
    soup = get_soup(url)
    next_page = None
    for left in soup.findAll('td', {'align':"left"}):
        next_page = _HOST+left.find_all('a', href=True)[0]['href']

    return next_page


def key_word(keyword, max_entry = 10, 
             title_exact_match = False, 
             year_after = '', year_until = '', 
             search_author = '', publish_in = ''): #search by key word.
    
    if title_exact_match == True:
        keyword_list = keyword.split(' ')
        keyword = '\''+ keyword +'\'' #put quotation marks on keywords
            
    
    if search_author != '':
        author_list = search_author.split(' ')
        for name in author_list:
            keyword = keyword + ' author:'+name
    
    if publish_in !='':
        publish_list = publish_in.split(' ')
        for source in publish_list:
            keyword = keyword + ' source:'+source
        
            
    url = _HOST+_PUBSEARCH.format(requests.utils.quote(keyword)) # get url
    number_of_entry=0
    counter = 0
    turn_page = True
    entry = {}
    
    title = []
    abstract = []
    
    while turn_page == True:
        url = url + '&as_ylo='+year_after + '&as_yhi='+year_until
        soup=get_soup(url)
        number_of_results = soup.findAll('div', {'class':'gs_ab_mdw'})[1]
        for row in soup.findAll('div', {'class':'gs_r gs_or gs_scl'}):
            authors = []
            sources = []
            if counter == max_entry:
                break
            links = row.findAll('h3', {'class':'gs_rt'})[0]
            if links.find_all('a', href=True) != []:
                link = links.find_all('a', href=True)[0]['href']
            else:
                link = None
            abstract = row.findAll('div', {'class':'gs_rs'})[0].get_text(strip=True)
            item=soup.findAll('div', {'class':'gs_a'})[counter].get_text(strip=True)
            author_soup = soup.findAll('div', {'class':'gs_a'})[counter]
            
            if re.findall(r" - (\D+)", item) != []:
                sources.append(re.findall(r" - (\D+)", item)[0])
                
            if re.findall("(\d+) - ", item) != []:
                year = re.findall("(\d+) - ", item)[0]

            else:
                item = [words for segments in item for words in segments.split(', ')]
                item = [words for segments in item for words in segments.split(',')]
                year = item[-1]
                
            item = re.findall(r"(.+?) - ", item)
            item = [words for segments in item for words in segments.split(', ')]
            item = [words for segments in item for words in segments.split(',')]

            
            for segment in item:
                if '- ' in segment:
                    segment = segment.split('- ')
                    name = re.sub('\xa0','',segment[0])
                    authors.append(name)
                    if segment[1].isnumeric() == False:
                        sources.append(segment[1])
                else:
                    source = None
                    if segment.isnumeric() == False:
                        authors.append(segment)
            
            for author_link in author_soup.find_all('a', href=True):
                author_info = []
                author_name = author_link.get_text(strip=True)
                author_id = re.findall("user=(.+?)&amp;hl", str(author_link))
                author_info = [author_name, author_id[0]]
                if author_info != []: 
                    for i in range(len(authors)):
                        if authors[i] == author_info[0]:
                            authors[i] = author_info
                        
                    
            title = soup.findAll('h3', {'class':'gs_rt'})[counter].get_text(strip=True)
            entry[number_of_entry] = {'authors': authors, 'article':title, 'url':link, 
                                      'sources':sources, 'year':year, 'abstract':abstract}
            
            number_of_entry+=1
            counter +=1

        if counter < max_entry:
            url = get_next_page(url)
            if url == None:
                turn_page = False
            max_entry = max_entry - counter
            counter = 0

        else:
            turn_page = False

    print(number_of_results.get_text(strip=True)) #print out how many results found
    return entry

keyword = key_word('Cultural Revolution', max_entry = 20, title_exact_match = True)

Page 2 of about 3,010,000 results (0.03sec)


In [252]:
def find_class_name(soup, tag, text):
    elements = soup.find_all(tag)
    for element in elements:
        if 'class' in element.attrs and text in element.attrs['class'][0]:
            return element.attrs['class'][0]
        else:
            return None

def scholar_search(name, max_entry = 10):
    url = _HOST+_AUTHSEARCH.format(requests.utils.quote(name)) #get url
     
    counter = 0
    turn_page = True
    number_of_entry=0
    counter = 0
    turn_page = True
    entry = {}
    authors = []
    title = []
    abstract = []
    
    while turn_page == True:

        soup = get_soup(url)
        number_of_results = soup.findAll('div', {'class':'gs_ab_mdw'})[1]

        for row in soup.find_all('div', 'gsc_1usr'):
            if counter == max_entry:
                break
            author_id = re.findall(_CITATIONAUTHRE, row('a')[0]['href'])[0]
            author_name = soup.find('h3', class_=find_class_name(soup, 'h3', 'name')).text
            for author_info in row.findAll('div', {'class':'gs_ai_t'}):
                citedby=author_info.findAll('div', {'class':'gs_ai_cby'})[0].get_text(strip=True)
                citedby = re.findall('\d+', citedby)[0]
                affiliation = author_info.findAll('div', {'class':'gs_ai_aff'})[0].get_text(strip=True)
                feilds = [i.text.strip()  for i in
                           author_info.findAll('a', {'class':'gs_ai_one_int'})]
                
            
            entry[number_of_entry] = {'author_id':author_id, 
                                      'name':author_name, 
                                      'affiliation':affiliation, 
                                      'feilds':feilds, 
                                      'citedby':citedby}
            number_of_entry+=1
            counter +=1
            
            
        if counter < max_entry:
            next_page = soup.find(class_='gs_btnPR gs_in_ib gs_btn_half gs_btn_lsb gs_btn_srt gsc_pgn_pnx')
            if next_page and 'disabled' not in next_page.attrs:
                url = next_page['onclick'][17:-1]
                url = _HOST + codecs.getdecoder("unicode_escape")(url)[0]
                soup = get_soup(url)
            else:
                turn_page = False
            max_entry = max_entry - counter
            counter = 0

        else:
            turn_page = False
    
    print(number_of_results.get_text(strip=True)) # print out how many results found
    return entry
scholar = scholar_search('Jian Chen') 

In [253]:
scholar

{0: {'author_id': 'piDcgOwAAAAJ',
  'name': 'Jian Chen',
  'affiliation': 'Associate Professor of Institute of Electronics, Chinese Academy of Sciences',
  'feilds': ['Microfluidics'],
  'citedby': '40445'},
 1: {'author_id': 'GuLkXX4AAAAJ',
  'name': 'Jian Chen',
  'affiliation': 'University of Leicester',
  'feilds': ['vibration',
   'acoustics',
   'bionic engineering',
   'machine condition monitoring',
   'and structural health monitoring'],
  'citedby': '22530'},
 2: {'author_id': 'EO2PG-8AAAAJ',
  'name': 'Jian Chen',
  'affiliation': 'Chair Professor of Ocean Eng., Southern University of Science and Technology',
  'feilds': ['Civil Engineering',
   'Structural Engineering',
   'Structures',
   'FRP',
   'DEM'],
  'citedby': '11364'},
 3: {'author_id': 'W6JT9HsAAAAJ',
  'name': 'Jian Chen',
  'affiliation': 'Associate Professor, Medical College of Georgia, Augusta University',
  'feilds': [],
  'citedby': '11212'},
 4: {'author_id': 'gA1_RnwAAAAJ',
  'name': 'Jian Chen',
  'affi

In [257]:
keyword

{0: {'authors': ['A Wallace-Hadrill'],
  'article': "[BOOK][B]Rome'scultural revolution",
  'url': 'https://pdfs.semanticscholar.org/44aa/d11dd73414876697092e21a8cd83ea926589.pdf',
  'sources': ['pdfs.semanticscholar.org'],
  'year': '2008',
  'abstract': "The period of Rome's imperial expansion, the late republic and ear- lier empire, saw transformationsof its society, culture and identity. Drawing equally on archaeological and literary evidence, thisbook offers an original and provocative interpretation of these changes. Moving from recent debates\xa0…"},
 1: {'authors': ['S Hite'],
  'article': '[BOOK][B]Women and love: Acultural revolutionin progress',
  'url': 'https://pdfs.semanticscholar.org/9003/cc0d11d62e74d7e36892c89c9d36d7e71e54.pdf',
  'sources': ['pdfs.semanticscholar.org'],
  'year': '1989',
  'abstract': 'The Hite Report women and love:: acultural revolutionin progress. The Hite report: women andlove: acultural revolutionin progress. User Review - Not Available - Book Ve

In [265]:
def export_csv(dict_data):
    
    try:
        csv_columns = dict_data[0].keys()
        csv_file = input('please enter a name for the CSV file')
        csv_file = csv_file+'.csv'
        with open(csv_file, 'w') as csvfile:
            writer = csv.DictWriter(csvfile, fieldnames=csv_columns)
            writer.writeheader()
            for data in dict_data:
                writer.writerow(data)
    except IOError:
        print("I/O error")

In [ ]:
export_csv(keyword)